# [BoTorch with Ax](https://botorch.org/tutorials/custom_botorch_model_in_ax)

> BoTorch makes no particular assumptions on what kind of model is being used, so long as is able to produce *samples from a posterior over outputs given an input x*.

> BoTorch abstracts away from the particular form of the posterior by providing a simple Posterior API that **only requires** implementing an **rsample()** method for sampling from the posterior.


## Simple gpytorch Exact GP Model
... with RBF(+ARD) kernel to infer homoskedastic noise level

[ARD:Automatic Relevance Determination](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ard.html)

----

### Radial Basis Function (RBF) kernel
One of the most basic form of kernel that measures similarity between two.
![RBF_eq](./RBF_eq.jpeg)
its similarity to Gaussian distribution is one of the reason for its popularity. 

------

####  [Gaussian MLE - derivation](http://jrmeyer.github.io/machinelearning/2017/08/18/mle.html)

-----

### Recommendation by GPyTorch
"If you don’t know what kernel to use, we recommend that you start out with a `gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel)`"


ConstantMean() is a safe bet as it converges to a constant as x -> infinity. In other words, the model will not make a dangerous guess where data points don't exist (near infinity). [Why mean of GP is uninteresting?](https://stats.stackexchange.com/questions/222238/why-is-the-mean-function-in-gaussian-process-uninteresting)

* Alternate to ExactGP are Approximate GP and Deep GP.

In [26]:
from botorch.models.gpytorch import GPyTorchModel
from gpytorch.distributions import MultivariateNormal
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.priors import GammaPrior


class SimplecCustomGP(ExactGP, GPyTorchModel):
    
    _num_outputs = 1
    
    def __init__(self, train_X, train_y):
        super().__init__(train_X, train_y.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=RBFKernel(ard_num_dims=train_X.shape[-1])) 
        self.to(train_X) # make sure dtype and device are matched to train_X data.
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

.forward() is the method called during the (PyTorch) training loop to get the output, or the posterior. 

### Python NOTE: Method Resolutio Order (MRO)
determines the order of inherited methods (in a multiple inheritance case)  
Note that `__mro__()` method is only visible for a class, not for a class instance.

### Define a factory function to be used with Ax's BotorchModel

In Ax's `BoTorchModel`, different components of Bayesian optimization (model generation & fitting; acquisition function; optimizing model and acq. function)are exposed via functional API. 

A factory function can be passed to replace the default component.  

The function MUST return a botorch `Model` object. What happens inside is up to you.

In [22]:
from botorch.fit import fit_gpytorch_model

def _get_and_fit(Xs, Ys, **kwards):
    """
    Return model
    """
    model = SimplecCustomGP(Xs[0], Ys[0])
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    return model

For a custom (non-GP) model, you need to build your own fitting loop.  
(https://botorch.org/tutorials/fit_model_with_torch_optimizer)

## Now, Ax part

We will optimize [Branin function](https://sites.google.com/site/gotestfunctions/multimodal-function-list/function-6-branin), which is a popular benchmark function for optimization techniques. i.e., has a quite complicated surface.
![branin](branin_fn.png)

Global optimum: f(xi) = 0.39788735772973816 for X = [-π , 12.275] or X = [π , 2.275] or [9.42478, 2.475]

In [8]:
import random
import numpy as np

def branin(param, *args):
    x1, x2 = param["x1"], param["x2"]
    y = (x2 - 5.1 / (4*np.pi**2) * x1**2 + 5 * x1 / np.pi -6)**2 + 10*(1- 1/(8*np.pi)) * np.cos(x1) + 10
    # Add noise
    y += random.normalvariate(0, 0.1) 
    return {"branin": (y, 0.0)}

### Define search space

In [12]:
from ax import ParameterType, RangeParameter, SearchSpace

search_space = SearchSpace(
    parameters=[
        RangeParameter(
            name='x1', parameter_type=ParameterType.FLOAT, lower=-5, upper=10),
        RangeParameter(
            name='x2', parameter_type=ParameterType.FLOAT, lower=0, upper=15),
    ]
)

### setup an 'experiment'

In [27]:
from ax import SimpleExperiment

exp = SimpleExperiment(
    name='test_branin',
    search_space=search_space,
    evaluation_function=branin,
    objective_name='branin',
    minimize=True,)

### Make up fake initial OBS.

[Sobol generator](https://medium.com/@antoine_savine/sobol-sequence-explained-188f422b246b)
![sobol](Sobol.png)
left: random, right: Sobol — Modern Computational Finance (Antoine Savine, Wiley, 2018), page 205

In [31]:
from ax.modelbridge import get_sobol

sobol = get_sobol(exp.search_space)
exp.new_batch_trial(generator_run=sobol.gen(5))

BatchTrial(experiment_name='test_branin', index=6, status=TrialStatus.CANDIDATE)

### Botorch model into the experiment

In [30]:
from ax.modelbridge.factory import get_botorch

for i in range(5):
    print(f"Running optimization batch {i+1}/5...")
    model = get_botorch(
        # exp carries information about search space and objective function.
        experiment = exp, 
        data = exp.eval(),
        search_space=exp.search_space,
        model_constructor=_get_and_fit
    )
    batch = exp.new_trial(generator_run=model.gen(1))
        
print("done!")

Running optimization batch 1/5...
Running optimization batch 2/5...
Running optimization batch 3/5...
Running optimization batch 4/5...
Running optimization batch 5/5...
done!
